# Merge All Datasets

In [2]:
import pandas as pd
import numpy as np

### 1. Load Data

### _parse dates_

- csv를 불러올 때, parse_dates을 적용하면 날짜 데이터를 파싱하고, 각 컬럼으로 분리, 할당할 수 있다.(아래 참조)
- 날짜 데이터를 포함한 dataset (train, weather)데이터에는 모두 적용해서, groupby와 pivot에 유용하게 사용하자.

In [3]:
# train as "sales" data
sales = pd.read_csv('train.csv', parse_dates=['date'])
print(sales.shape)
sales.head()

(4617600, 4)


,date,store_nbr,item_nbr,units
0,2012-01-01,1,1,0
1,2012-01-01,1,2,0
2,2012-01-01,1,3,0
3,2012-01-01,1,4,0
4,2012-01-01,1,5,0


In [4]:
# Keys : Key(station_nbr, store_nbr)
keys = pd.read_csv('key.csv')
print(keys.shape)
keys.head()

(45, 2)


,store_nbr,station_nbr
0,1,1
1,2,14
2,3,7
3,4,9
4,5,12


In [5]:
# Weather Log
weather = pd.read_csv('weather.csv',parse_dates=['date'])
print(weather.shape)
weather.head()

(20517, 20)


,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0
3,4,2012-01-01,63,47,55,4,28,43,10,0,0728,1742,,0.0,0.00,29.79,30.48,8.0,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8


In [6]:
df_1 = pd.merge(weather,keys)
print(df_1.shape)
df_1.head()

(46392, 21)


,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,store_nbr
0,1,2012-01-01,52,31,42,M,36,40,23,0,...,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6,1
1,1,2012-01-02,50,31,41,M,26,35,24,0,...,-,,M,0.01,29.44,29.62,9.8,24,10.3,1
2,1,2012-01-03,32,11,22,M,4,18,43,0,...,-,,M,0.00,29.67,29.87,10.8,31,11.6,1
3,1,2012-01-04,28,9,19,M,-1,14,46,0,...,-,,M,0.00,29.86,30.03,6.3,27,8.3,1
4,1,2012-01-05,38,25,32,M,13,25,33,0,...,-,,M,0.00,29.67,29.84,6.9,25,7.8,1


In [7]:
df_1 = pd.merge(df_1, sales)
print(df_1.shape)
df_1.tail()

(4617600, 23)


,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,store_nbr,item_nbr,units
4617595,5,2014-10-31,M,M,M,M,37,M,M,M,...,M,M,M,30.01,10.8,05,M,35,107,0
4617596,5,2014-10-31,M,M,M,M,37,M,M,M,...,M,M,M,30.01,10.8,05,M,35,108,0
4617597,5,2014-10-31,M,M,M,M,37,M,M,M,...,M,M,M,30.01,10.8,05,M,35,109,0
4617598,5,2014-10-31,M,M,M,M,37,M,M,M,...,M,M,M,30.01,10.8,05,M,35,110,0
4617599,5,2014-10-31,M,M,M,M,37,M,M,M,...,M,M,M,30.01,10.8,05,M,35,111,0


In [8]:
dates = df_1['date'].dt
df_2 = pd.DataFrame()
df_2['year'] = dates.year
df_2['month'] = dates.month
df_2['day'] = dates.day
df_2['station_nbr'] = df_1['station_nbr']
df_2['store_nbr'] = df_1['store_nbr']
df_2['units'] = df_1['units']

In [9]:
double_sort = df_2.groupby(by=['year','month','store_nbr','station_nbr']).agg('sum').reset_index()

In [10]:
final_sample = double_sort.sort_values(by=['year','month'])
final_sample = final_sample.drop(columns=['day'])

In [11]:
final_sample.head()

,year,month,store_nbr,station_nbr,units
0,2012,1,1,1,1350
1,2012,1,2,14,6676
2,2012,1,3,7,5329
3,2012,1,4,9,2845
4,2012,1,5,12,1705


### Check `sales` &  `final_sample`

final_sample의 월 별 units의 총합과 sales 데이터로 도출한 월 별 units의 총합은 일치하는 것을 확인했다.

In [12]:
sales_2 = pd.DataFrame()
dates = sales['date'].dt
sales_2['year'] = dates.year
sales_2['month'] = dates.month
sales_2['day'] = dates.day
sales_2['store_nbr'] = sales['store_nbr']
sales_2['units'] = sales['units']

sales_2.head()

,year,month,day,store_nbr,units
0,2012,1,1,1,0
1,2012,1,1,1,0
2,2012,1,1,1,0
3,2012,1,1,1,0
4,2012,1,1,1,0


In [14]:
sales_2_sort = sales_2.groupby(by=['year','month','store_nbr']).agg('sum').reset_index()
sales_2_sort = sales_2_sort.drop(columns=['day'])
sales_2_sort.head()

,year,month,store_nbr,units
0,2012,1,1,1350
1,2012,1,2,6676
2,2012,1,3,5329
3,2012,1,4,2845
4,2012,1,5,1705


# final_sample로 작업하면 됨

In [103]:
# ex) station 별로 처리하고 싶다
station = final_sample.pivot_table(index=['station_nbr','year','month'],values='units')
station.tail()

units
station_nbr year month        
20          2014 6      4772.0
                 7      3724.0
                 8      4095.0
                 9      4569.0
                 10     2664.0

In [104]:
# ex) 월별로 처리하고 싶다
month = final_sample.pivot_table(index=['year','month','station_nbr','store_nbr'],values='units')
month.tail()

units
year month station_nbr store_nbr       
2014 10    17          31          1763
                       34          1830
           18          36          1695
           19          30          2151
           20          17          2664

### 더 세부적인 조합을 위해 final_sample 에 day, codesum 데이터 추가 (final_sample_2)

In [25]:
df_2['codesum'] = df_1['codesum']
df_2.head()

,year,month,day,station_nbr,store_nbr,units,codesum
0,2012,1,1,1,1,0,RA FZFG BR
1,2012,1,1,1,1,0,RA FZFG BR
2,2012,1,1,1,1,0,RA FZFG BR
3,2012,1,1,1,1,0,RA FZFG BR
4,2012,1,1,1,1,0,RA FZFG BR


In [26]:
final_sample_2 = df_2.groupby(by=['year','month','day', 'store_nbr', 'station_nbr', 'codesum']).agg('sum').reset_index()
final_sample_2

,year,month,day,store_nbr,station_nbr,codesum,units
0,2012,1,1,1,1,RA FZFG BR,32
1,2012,1,1,2,14,,406
2,2012,1,1,3,7,,326
3,2012,1,1,4,9,UP,82
4,2012,1,1,5,12,FG+ FG BR,41
5,2012,1,1,6,14,,130
6,2012,1,1,7,6,,155
7,2012,1,1,8,4,,42
8,2012,1,1,9,17,,211
9,2012,1,1,10,12,FG+ FG BR,133
